In [0]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import networkx as nx
from scipy.stats import poisson, norm, uniform



#Function to add cities as nodes
def add_city(graph,name,area,population,cases,neighbours):


  #neighbours is a list of tuples. Each tuple contains the name of the city,
  #the area, the population, its neighbours and the strength between each of its
  #neighbours (weight). See the second cell for the format of the data

  #subtract the cases from the population to get the uninfected population
  population -= cases
  population_density = population/area

  #Add node to graph
  graph.add_node(name,population=population,
                 population_density=population_density,
                 cases=cases)
  
  #Add edges between the neighbours
  graph.add_weighted_edges_from(neighbours)

#Function that simulates a day
def simulate_day(graph,lambda_,recovery):

  #Loop through nodes
  for node in graph.nodes:

    #Loop through each case and sample from a poisson distribution to see how
    #many people the case infected
    for people in range(graph.nodes[node]['cases']):
      spread = poisson.rvs(lambda_)
      graph.nodes[node]['cases'] += spread
      graph.nodes[node]['population'] -= spread


    #Multiplier effect due to density of city
    spread = int(round(graph.nodes[node]['population_density'] * abs(norm.rvs(0,0.001) * graph.nodes[node]['cases'])))
    graph.nodes[node]['cases'] += spread
    graph.nodes[node]['population'] -= spread  

    #Small chance of recovery in people
    recovered = 0
    recovery_rate = round(abs(norm.rvs(recovery,0.05)))
    for people in range(graph.nodes[node]['cases']):
      if uniform.rvs() < recovery_rate:
        recovered+=1
    graph.nodes[node]['cases'] -= recovered
    graph.nodes[node]['population'] += recovered


    #Loop through each edge of each node to simulate flow of people
    for edges in graph.adj[node]:
      #get the weight
      weight =  list(graph[node][edges]['weight'].values())[0]

      #get expected flow of traffic from the city
      expected_flow = round((weight + abs(norm.rvs(0,0.01))) *  (graph.nodes[node]['cases']+graph.nodes[node]['population']))

      #check how many infected people travel
      samples = uniform.rvs(size=int(expected_flow))
      corona_flow = 0
      for i in samples:
        if i < graph.nodes[node]['cases']/graph.nodes[node]['population']:
          corona_flow += 1
      graph.nodes[node]['cases'] -= corona_flow
      graph.nodes[edges]['cases'] += corona_flow







# Example for the district of Sindh in Pakistan

In [3]:


#Make sindh graph
sindh = nx.Graph()

#Add cities/districts

#data was taken from https://en.wikipedia.org/wiki/Districts_of_Sindh,_Pakistan

add_city(sindh, 'Badin',6470,1804516,10, [("Badin",'Thatta',{'weight':abs(norm.rvs(0.001,0.001))}),("Badin",'Tando Muhammad Khan',{'weight':abs(norm.rvs(0.001,0.001))}),
                                          ("Badin",'Hyderabad',{'weight':abs(norm.rvs(0.001,0.001))}), ("Badin",'Tando Allahyar',{'weight':abs(norm.rvs(0.001,0.001))}),
                                          ("Badin",'Mirpur Khas',{'weight':abs(norm.rvs(0.001,0.001))}), ("Badin",'Tharparker',{'weight':abs(norm.rvs(0.001,0.001))})] )
add_city(sindh, 'Dadu',8034,1550266,10, [("Dadu",'Qambar Shahdadkot',{'weight':abs(norm.rvs(0.001,0.001))}),("Dadu",'Larkana',{'weight':abs(norm.rvs(0.001,0.001))}),
                                         ("Dadu",'Larkana',{'weight':abs(norm.rvs(0.001,0.001))}),("Dadu",'Naushahro Firoz',{'weight':abs(norm.rvs(0.001,0.001))}),
                                         ("Dadu",'Jamshoro',{'weight':abs(norm.rvs(0.001,0.001))})])
add_city(sindh, 'Ghotki',6506,1647239,10, [("Ghotki",'Kashmore',{'weight':abs(norm.rvs(0.001,0.001))}), ("Ghotki",'Sukkur',{'weight':abs(norm.rvs(0.001,0.001))})] )
add_city(sindh, 'Hyderabad',1022,2201079,10, [("Hyderabad",'Jamshoro',{'weight':abs(norm.rvs(0.001,0.001))}), ("Hyderabad",'Matiari',{'weight':abs(norm.rvs(0.001,0.001))}),
                                              ("Hyderabad",'Sanghar',{'weight':abs(norm.rvs(0.001,0.001))}), ("Hyderabad",'Tando Allahyar',{'weight':abs(norm.rvs(0.001,0.001))}),
                                              ("Hyderabad",'Badin',{'weight':abs(norm.rvs(0.001,0.001))}),("Hyderabad",'Tando Muhammad Khan',{'weight':abs(norm.rvs(0.001,0.001))}),
                                              ("Hyderabad",'Thatta',{'weight':abs(norm.rvs(0.001,0.001))})] )
add_city(sindh, 'Jacobabad',2771,1006297,10, [("Jacobabad",'Kashmore',{'weight':abs(norm.rvs(0.001,0.001))}),("Jacobabad",'Shikarpur',{'weight':abs(norm.rvs(0.001,0.001))}),
                                              ("Jacobabad",'Larkana',{'weight':abs(norm.rvs(0.001,0.001))}),("Jacobabad",'Qambar Shahdadkot',{'weight':abs(norm.rvs(0.001,0.001))})] )
add_city(sindh, 'Jamshoro',11250,993142,10, [ ("Jamshoro",'Dadu',{'weight':abs(norm.rvs(0.001,0.001))}),  ("Jamshoro",'Naushahro Firoz',{'weight':abs(norm.rvs(0.001,0.001))}),
                                              ("Jamshoro",'Nawabshah',{'weight':abs(norm.rvs(0.001,0.001))}), ("Jamshoro",'Matiari',{'weight':abs(norm.rvs(0.001,0.001))}),
                                              ("Jamshoro",'Hyderabad',{'weight':abs(norm.rvs(0.001,0.001))}), ("Jamshoro",'Tando Muhammad Khan',{'weight':abs(norm.rvs(0.001,0.001))}),
                                              ("Jamshoro",'Thatta',{'weight':abs(norm.rvs(0.001,0.001))}), ("Jamshoro",'Karachi',{'weight':abs(norm.rvs(0.001,0.001))})] )
add_city(sindh, 'Karachi',3672,17144157,10, [ ("Karachi",'Jamshoro',{'weight':abs(norm.rvs(0.001,0.001))}),  ("Karachi",'Thatta',{'weight':abs(norm.rvs(0.001,0.001))})] )
add_city(sindh, 'Kashmore',2551,1089169,10, [ ("Kashmore",'Ghotki',{'weight':abs(norm.rvs(0.001,0.001))}),("Kashmore",'Sukkur',{'weight':abs(norm.rvs(0.001,0.001))}),
                                             ("Kashmore",'Shikarpur',{'weight':abs(norm.rvs(0.001,0.001))}),("Kashmore",'Jacobabad',{'weight':abs(norm.rvs(0.001,0.001))})] )
add_city(sindh, 'Khairpur',15925,2405523,10, [("Khairpur",'Larkana',{'weight':abs(norm.rvs(0.001,0.001))}),("Khairpur",'Sukkur',{'weight':abs(norm.rvs(0.001,0.001))}),
                                              ("Khairpur",'Shikarpur',{'weight':abs(norm.rvs(0.001,0.001))}), ("Khairpur",'Nawabshah',{'weight':abs(norm.rvs(0.001,0.001))}),
                                              ("Khairpur",'Sanghar',{'weight':abs(norm.rvs(0.001,0.001))}), ("Khairpur",'Naushahro Firoz',{'weight':abs(norm.rvs(0.001,0.001))})] )
add_city(sindh, 'Larkana',1906,1524391,10, [("Larkana",'Shikarpur',{'weight':abs(norm.rvs(0.001,0.001))}),("Larkana",'Jacobabad',{'weight':abs(norm.rvs(0.001,0.001))}),
                                            ("Larkana",'Qambar Shahdadkot',{'weight':abs(norm.rvs(0.001,0.001))}),("Larkana",'Dadu',{'weight':abs(norm.rvs(0.001,0.001))}),
                                            ("Larkana",'Naushahro Firoz',{'weight':abs(norm.rvs(0.001,0.001))}),("Larkana",'Khairpur',{'weight':abs(norm.rvs(0.001,0.001))})] )
add_city(sindh, 'Matiari',1459,769349,10, [("Matiari",'Nawabshah',{'weight':abs(norm.rvs(0.001,0.001))}),("Matiari",'Sanghar',{'weight':abs(norm.rvs(0.001,0.001))}),
                                           ("Matiari",'Hyderabad',{'weight':abs(norm.rvs(0.001,0.001))}), ("Matiari",'Jamshoro',{'weight':abs(norm.rvs(0.001,0.001))})] )
add_city(sindh, 'Mirpur Khas',3319,1505876,10, [("Mirpur Khas",'Sanghar',{'weight':abs(norm.rvs(0.001,0.001))}), ("Mirpur Khas",'Umerkot',{'weight':abs(norm.rvs(0.001,0.001))}),
                                                ("Mirpur Khas",'Tharparker',{'weight':abs(norm.rvs(0.001,0.001))}), ("Mirpur Khas",'Tando Allahyar',{'weight':abs(norm.rvs(0.001,0.001))}),
                                                ("Mirpur Khas",'Badin',{'weight':abs(norm.rvs(0.001,0.001))})] )
add_city(sindh, 'Naushahro Firoz',2027,1612373,10, [("Naushahro Firoz",'Dadu',{'weight':abs(norm.rvs(0.001,0.001))}),("Naushahro Firoz",'Larkana',{'weight':abs(norm.rvs(0.001,0.001))}),
                                                    ("Naushahro Firoz",'Khairpur',{'weight':abs(norm.rvs(0.001,0.001))}),("Naushahro Firoz",'Nawabshah',{'weight':abs(norm.rvs(0.001,0.001))}),
                                                    ("Naushahro Firoz",'Jamshoro',{'weight':abs(norm.rvs(0.001,0.001))})])
add_city(sindh, 'Nawabshah',4618,2012847,10, [("Nawabshah",'Khairpur',{'weight':abs(norm.rvs(0.001,0.001))}),("Nawabshah",'Sanghar',{'weight':abs(norm.rvs(0.001,0.001))}),
                                              ("Nawabshah",'Naushahro Firoz',{'weight':abs(norm.rvs(0.001,0.001))}),("Nawabshah",'Matiari',{'weight':abs(norm.rvs(0.001,0.001))}),
                                              ("Nawabshah",'Jamshoro',{'weight':abs(norm.rvs(0.001,0.001))})] )
add_city(sindh, 'Qambar Shahdadkot',5599,1341042,10, [("Qambar Shahdadkot",'Larkana',{'weight':abs(norm.rvs(0.001,0.001))}),("Qambar Shahdadkot",'Dadu',{'weight':abs(norm.rvs(0.001,0.001))}),
                                                      ("Qambar Shahdadkot",'Jacobabad',{'weight':abs(norm.rvs(0.001,0.001))})] )
add_city(sindh, 'Sanghar',10259,2057057	,10, [("Sanghar",'Khairpur',{'weight':abs(norm.rvs(0.001,0.001))}),("Sanghar",'Matiari',{'weight':abs(norm.rvs(0.001,0.001))}),
                                              ("Sanghar",'Nawabshah',{'weight':abs(norm.rvs(0.001,0.001))}),("Sanghar",'Hyderabad',{'weight':abs(norm.rvs(0.001,0.001))}),
                                              ("Sanghar",'Mirpur Khas',{'weight':abs(norm.rvs(0.001,0.001))}),("Sanghar",'Tando Muhammad Khan',{'weight':abs(norm.rvs(0.001,0.001))}),
                                              ("Sanghar",'Umerkot',{'weight':abs(norm.rvs(0.001,0.001))})] )
add_city(sindh, 'Shikarpur',2577,1231481,10, [("Shikarpur",'Jacobabad',{'weight':abs(norm.rvs(0.001,0.001))}),("Shikarpur",'Kashmore',{'weight':abs(norm.rvs(0.001,0.001))}),
                                              ("Shikarpur",'Ghotki',{'weight':abs(norm.rvs(0.001,0.001))}),("Shikarpur",'Larkana',{'weight':abs(norm.rvs(0.001,0.001))}),
                                              ("Shikarpur",'Sukkur',{'weight':abs(norm.rvs(0.001,0.001))}),("Shikarpur",'Khairpur',{'weight':abs(norm.rvs(0.001,0.001))})] )
add_city(sindh, 'Sukkur',5216,1487903,10, [("Sukkur",'Ghotki',{'weight':abs(norm.rvs(0.001,0.001))}),("Sukkur",'Kashmore',{'weight':abs(norm.rvs(0.001,0.001))}),
                                           ("Sukkur",'Shikarpur',{'weight':abs(norm.rvs(0.001,0.001))}),("Sukkur",'Khairpur',{'weight':abs(norm.rvs(0.001,0.001))})] )
add_city(sindh, 'Tando Allahyar',1573,836887,10, [("Tando Allahyar",'Sanghar',{'weight':abs(norm.rvs(0.001,0.001))}),("Tando Allahyar",'Mirpur Khas',{'weight':abs(norm.rvs(0.001,0.001))}),
                                                  ("Tando Allahyar",'Badin',{'weight':abs(norm.rvs(0.001,0.001))}),("Tando Allahyar",'Hyderabad',{'weight':abs(norm.rvs(0.001,0.001))})] )
add_city(sindh, 'Tando Muhammad Khan',1814,677228,10, [("Tando Muhammad Khan",'Hyderabad',{'weight':abs(norm.rvs(0.001,0.001))}),("Tando Muhammad Khan",'Thatta',{'weight':abs(norm.rvs(0.001,0.001))}),
                                                         ("Tando Muhammad Khan",'Badin',{'weight':abs(norm.rvs(0.001,0.001))})] )
add_city(sindh, 'Tharparker',19808,1649661,10, [("Tharparker",'Umerkot',{'weight':abs(norm.rvs(0.001,0.001))}),("Tharparker",'Badin',{'weight':abs(norm.rvs(0.001,0.001))}),
                                                ("Tharparker",'Mirpur Khas',{'weight':abs(norm.rvs(0.001,0.001))})] )
add_city(sindh, 'Thatta',16404,1761784,10, [("Thatta",'Hyderabad',{'weight':abs(norm.rvs(0.001,0.001))}),("Thatta",'Jamshoro',{'weight':abs(norm.rvs(0.001,0.001))}),
                                            ("Thatta",'Karachi',{'weight':abs(norm.rvs(0.001,0.001))}),("Thatta",'Tando Muhammad Khan',{'weight':abs(norm.rvs(0.001,0.001))}),
                                            ("Thatta",'Badin',{'weight':abs(norm.rvs(0.001,0.001))})] )
add_city(sindh, 'Umerkot',5503,1073146,10, [("Umerkot",'Sanghar',{'weight':abs(norm.rvs(0.001,0.001))}),("Umerkot",'Mirpur Khas',{'weight':abs(norm.rvs(0.001,0.001))}),
                                            ("Umerkot",'Tharparker',{'weight':abs(norm.rvs(0.001,0.001))})] )


#Run the simulation for an X number of days. X is determined by how many times
#you run the loop
for i in range(1):
  simulate_day(sindh,1,0.1)

#city_est = [[] for x in range(23)]
  
total = 0
for i in sindh.nodes:
  total += sindh.nodes[i]['cases']
  print('Number of cases in ', i, ' are ', sindh.nodes[i]['cases'])

print('Total cases in Sindh are', total)



Number of cases in  Badin  are  22
Number of cases in  Thatta  are  24
Number of cases in  Tando Muhammad Khan  are  24
Number of cases in  Hyderabad  are  102
Number of cases in  Tando Allahyar  are  32
Number of cases in  Mirpur Khas  are  21
Number of cases in  Tharparker  are  20
Number of cases in  Dadu  are  18
Number of cases in  Qambar Shahdadkot  are  20
Number of cases in  Larkana  are  37
Number of cases in  Naushahro Firoz  are  55
Number of cases in  Jamshoro  are  21
Number of cases in  Ghotki  are  23
Number of cases in  Kashmore  are  44
Number of cases in  Sukkur  are  37
Number of cases in  Matiari  are  41
Number of cases in  Sanghar  are  25
Number of cases in  Jacobabad  are  25
Number of cases in  Shikarpur  are  62
Number of cases in  Nawabshah  are  41
Number of cases in  Karachi  are  40
Number of cases in  Khairpur  are  32
Number of cases in  Umerkot  are  18
Total cases in Sindh are 784
